This notebook implements a simple flow described in the following:

cotton - consume - Sew Gown - produce - gown

gown - transfer-custody - gown

gown - accept - Use Gown - modify - gown
              /
         work

gown - transfer-custody - gown

water - consume\
  gown - accept - Clean Gown - modify
soap - consume /                                              

In [ ]:
# Let's start with some lovely imports that should have been installed if not available by default
import json
import requests
import os
import urllib.parse
from zenroom import zenroom
from IPython.core.debugger import set_trace
import re
import base64
from datetime import datetime, timezone
import random
import plotly.graph_objects as go


In [ ]:
USE_CASE = 'isogowns'

# What endpoint are we talking to?
# debug
ENDPOINT = 'http://65.109.11.42:10000/api'
# ENDPOINT = 'http://zenflows-debug.interfacer.dyne.org'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'


In [ ]:
# This does not include transfers
SUPPORTED_ACTIONS = ['accept', 'cite', 'consume', 'modify', 'produce', 'use', 'work']
IN_PR_ACTIONS = ['accept', 'cite', 'consume', 'use', 'work']
OUT_PR_ACTIONS = ['modify', 'produce']
assert set(IN_PR_ACTIONS + OUT_PR_ACTIONS) == set(SUPPORTED_ACTIONS)


MAX_DEPTH = 100000000
AGENT_FRAG = """
    fragment agent on Agent {
        id
        name
    }
"""
LOCATION_FRAG = """
    fragment location on SpatialThing {
        id
        alt
        lat
        long
        mappableAddress
        name
        note
    }
"""
QUANTITY_FRAG = """
    fragment quantity on Measure {
      hasNumericalValue
      hasUnit {
        id
        label
        symbol
      }

    }
"""

RESOURCE_FRAG = """
    fragment resource on EconomicResource {
            id
            name
            onhandQuantity {
                ...quantity
            }
            accountingQuantity {
                ...quantity
            }
            primaryAccountable {
                ...agent
            }
            custodian {
                ...agent
            }
          }
"""


In [ ]:
# Test zenroom is correctly installed and running witht the following function
def generate_random_challenge():
    """
        This function calls zenroom to generate
        a random string to be used as challenge
    """
    contract = """
        rule check version 1.0.0
        Given nothing
        When I create the random object of '512' bits
        and I rename the 'random_object' to 'challenge'
        Then print 'challenge'
    """

    try:
        result = zenroom.zencode_exec(contract)
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

    res_json = json.loads(result.output)

    print(f"Generated challenge: {res_json['challenge']}")

    return


In [ ]:
# This should print something like
# Generated challenge: MMQ1JSrCA7L4QNftLyaaSRunT4Z9+Rr2QkE+a+DWLEljtg6EroLbCj5VjLH+xba9Rv1D+3ncQHw5s/lH41IFJw==
generate_random_challenge()

In [ ]:
# we will save endpoint specific files since the data is saved on a particular endpoint
def get_filename(filename, ep=ENDPOINT, uc=USE_CASE):
    pattern = r'http[s]?://'
    strp_endpoint = re.sub(pattern, '', ep)
    new_filename = filename.replace('.json',f'.{urllib.parse.quote_plus(strp_endpoint)}.{uc}.json')
    return new_filename

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []


file = get_filename('cred_users.json')
if os.path.isfile(file):
    with open(file,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['one'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "User One",
      "username": "user1_username",
      "email": "user1@example.org",
      "note": "me.user1.org"
    }
    users_data['two'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "User Two",
        "username": "user2",
        "email": "user2@example.org",
        "note" : "me.user2.org"
    }

    with open(file,'w') as f:
        json.dump(users_data, f)

file = get_filename('loc_users.json')
if os.path.isfile(file):
    with open(file,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['one'] = {
        "name": "OLVG",
        "lat": 52.35871773455108,
        "long": 4.916762398221842,
        "addr": "Oosterpark 9, 1091 AC Amsterdam",
        "note": "location.user1.org"
    }
    locs_data['two'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven",
        "note": "location.user2.org"
    }
    with open(file,'w') as f:
        json.dump(locs_data, f)

file = get_filename('units_data.json')
if os.path.isfile(file):
    with open(file,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)


file = get_filename('res_spec_data.json')
if os.path.isfile(file):
    with open(file,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}

# Function to show all the data
def show_data(users_data=users_data,locs_data=locs_data,res_data=res_data,units_data=units_data, 
              res_spec_data=res_spec_data, process_data=process_data, event_seq=event_seq):
    print("Users")
    print(json.dumps(users_data, indent=2))
    print("Locations")
    print(json.dumps(locs_data, indent=2))

    print("Resources")
    print(json.dumps(res_data, indent=2))
    print("Units")
    print(json.dumps(units_data, indent=2))

    print("Resource Specifications")
    print(json.dumps(res_spec_data, indent=2))

    print("Process Definitions")
    print(json.dumps(process_data, indent=2))

    print("Event sequence")
    print(json.dumps(event_seq, indent=2))
  

In [ ]:
# this function is equivalent to JSON.stringify in javascript, i.e. it does not add spaces
# Although it does not seem to matter as Zenroom removes spaces
def stringify(json_obj):
    return json.dumps(json_obj, separators=(',',':'))

In [ ]:
# Get the seed from the server to later generate the keypair
DEBUG_get_HMAC = False
def get_HMAC(email, endpoint, newUser=True):

    variables = {
        "firstRegistration": newUser,
        "userData": "{\"email\": \"" + email + "\"}"
    };

    
    payload = {
      "query": """mutation ($firstRegistration: Boolean!, $userData: String!){
  
        keypairoomServer(firstRegistration: $firstRegistration, userData: $userData)
      
      }""",
      "variables": variables
    }
    

    res = requests.post(endpoint, json=payload)
    if DEBUG_get_HMAC:
        print("Payload")
        print(payload)
        print("Variables")
        print(variables)
        print("Result")
        print(res)

    result = res.json()
    
    if DEBUG_get_HMAC:
        print("JSON")
        print(json.dumps(result, indent=2))

    if "errors" in result and len(result['errors']) > 0:
        for err in result['errors']:
            if err['message'] == "email exists":
                return get_HMAC(email, newUser=False)
    
    return result

In [ ]:
# if the HMAC is not in the conf files call the function to get it
def read_HMAC(file, users_data, user, endpoint):
    
    # this should not be possible since we initialize the data, but anyway
    if not f'{user}' in users_data:
        users_data[f'{user}'] = {}
        print("Warning this should not happen")

    user_data = users_data[f'{user}']

    # check we already have a credentials file with a HMAC
    if 'seedServerSideShard.HMAC' in user_data:
        print(f"Server HMAC available for {user_data['name']}")
        return
    if os.path.isfile(file):
        with open(file,'r') as f:
                tmp_users_data = json.loads(f.read())
                tmp_user_data = tmp_users_data[f'{user}'] 
                

    else:
        tmp_user_data = {}
        
    if 'seedServerSideShard.HMAC' not in tmp_user_data:
        res = get_HMAC(user_data['email'], endpoint)
        # print(res)
        # save the HMAC in the user data
        user_data['seedServerSideShard.HMAC'] = res['data']['keypairoomServer']

        # save data with HMAC
        with open(file,'w') as f:
            # Save the entire data structure
            json.dump(users_data, f)
    else:
        user_data['seedServerSideShard.HMAC'] = tmp_user_data['seedServerSideShard.HMAC']
        # no need to save since we read it from file
    

In [ ]:
# Read HMAC or get it from the server
endpt_filename = get_filename('cred_users.json')

read_HMAC(endpt_filename, users_data, 'one', endpoint=ENDPOINT)
read_HMAC(endpt_filename, users_data, 'two', endpoint=ENDPOINT)

In [ ]:
DEBUG_generate_keypair = False
# Generate the user keypair (and the mnemonic seed)
def generate_keypair(userdata):
    """
        This function calls zenroom to generate
        a keypair using the server-provided HMAC
    """
    contract = """
        Scenario 'ecdh': Create the key
        Scenario 'ethereum': Create key
        Scenario 'reflow': Create the key
        Scenario 'schnorr': Create the key
        Scenario 'eddsa': Create the key
        Scenario 'qp': Create the key


        # Loading the user name from data
        Given my name is in a 'string' named 'username'

        # Loading the answers from 3 secret questions. The user will have to pick the 3 challenges from a list 
        # and have to remember the questions - the order is not important cause Zenroom will sort alphabetically 
        # the data in input
        #
        # NOTE: the challenges will never be communicated to the server or to anybody else!
        Given I have a 'string dictionary' named 'userChallenges'

        # Loading the individual challenges, in order to have them hashed 
        # and the hashes OPTIONALLY stored by the server, to improve regeneration of the keypair
        Given I have a 'string' named 'whereParentsMet' in 'userChallenges'
        Given I have a 'string' named 'nameFirstPet' in 'userChallenges'
        Given I have a 'string' named 'whereHomeTown' in 'userChallenges'
        Given I have a 'string' named 'nameFirstTeacher' in 'userChallenges'
        Given I have a 'string' named 'nameMotherMaid' in 'userChallenges'

        # Loading the pbkdf received from the server, containing a signed hash of known data
        Given that I have a 'base64' named 'seedServerSideShard.HMAC' 

        # Save the backup for mnemonic dump, before factoring with the salt
        # it is shortened to 16 bytes by hashing sha512 the KDF and taking the first 16 bytes
        When I create the key derivation of 'userChallenges'
        and I create the hash of 'key derivation' using 'sha512'
        and I split the leftmost '16' bytes of 'hash'
        and I delete the 'key derivation'
        and I delete the 'hash'
        and I rename the 'leftmost' to 'seed'

        # Hash again the user's challenges with salt for the seed root
        When I rename 'seedServerSideShard.HMAC' to 'salt'
        and I create the key derivation of 'seed' with password 'salt'
        and I rename the 'key derivation' to 'seed.root'

        # In the following flow the order should NOT be changed

        When I create the hash of 'seed.root'
        When I rename the 'hash' to 'seed.ecdh'

        When I create the hash of 'seed.ecdh'
        When I rename the 'hash' to 'seed.eddsa'

        When I create the hash of 'seed.eddsa'
        When I rename the 'hash' to 'seed.ethereum'

        When I create the hash of 'seed.ethereum'
        When I rename the 'hash' to 'seed.reflow'

        When I create the hash of 'seed.reflow'
        When I rename the 'hash' to 'seed.schnorr'

        # end of the sorted creation flow

        When I create the ecdh key with secret key 'seed.ecdh'
        When I create the eddsa key with secret key 'seed.eddsa'
        When I create the ethereum key with secret key 'seed.ethereum'
        When I create the reflow key with secret key 'seed.reflow'
        When I create the schnorr key with secret key 'seed.schnorr'

        When I create the ecdh public key
        When I create the eddsa public key
        When I create the ethereum address
        When I create the reflow public key
        When I create the schnorr public key

        # Creating the hashes of the single challenges, to OPTIONALLY help 
        # regeneration of the keypair

        When I create the 'base64 dictionary'
        and I rename the 'base64 dictionary' to 'hashedAnswers'

        When I create the key derivation of 'whereParentsMet'
        and I rename the 'key derivation' to 'whereParentsMet.kdf'
        When I insert 'whereParentsMet.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameFirstPet'
        and I rename the 'key derivation' to 'nameFirstPet.kdf'
        When I insert 'nameFirstPet.kdf' in 'hashedAnswers'

        When I create the key derivation of 'whereHomeTown'
        and I rename the 'key derivation' to 'whereHomeTown.kdf'
        When I insert 'whereHomeTown.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameFirstTeacher'
        and I rename the 'key derivation' to 'nameFirstTeacher.kdf'
        When I insert 'nameFirstTeacher.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameMotherMaid'
        and I rename the 'key derivation' to 'nameMotherMaid.kdf'
        When I insert 'nameMotherMaid.kdf' in 'hashedAnswers'


        # This prints the keyring
        Then print the 'keyring' 

        # this prints the hashes of the challenges
        # Then print the 'hashedAnswers'

        # This prints the seed for the private keys as mnemonic 
        Then print the 'seed' as 'mnemonic'

        Then print the 'ecdh public key'
        Then print the 'eddsa public key'
        Then print the 'ethereum address'
        Then print the 'reflow public key'
        Then print the 'schnorr public key'
    """
    
    data = json.dumps(userdata)

    try:
        result = zenroom.zencode_exec(contract, data=data)
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

    if DEBUG_generate_keypair:
        print(f'result: {result}')

    res_json = json.loads(result.output)

    if DEBUG_generate_keypair:
        print(f"Generated keypair data: {json.dumps(res_json, indent=2)}")

    return res_json


In [ ]:
# Read the keypair from conf files or call the function to generate it
def read_keypair(file, users_data, user):
    
    # this should not be possible since we initialize the data, but anyway
    if not f'{user}' in users_data:
        users_data[f'{user}'] = {}
        print("Warning this should not happen")
        
    user_data = users_data[f'{user}']
    
    if ('seed' in user_data and 'eddsa_public_key' in user_data and \
            'keyring' in user_data and 'eddsa' in user_data['keyring']):
        print(f"Keypair available for {user_data['name']}")
        return
        
    if os.path.isfile(file):
        with open(file,'r') as f:
                tmp_users_data = json.loads(f.read())
                tmp_user_data = tmp_users_data[f'{user}'] 
    else:
        tmp_user_data = {}

    if not ('seed' in tmp_user_data and 'eddsa_public_key' in tmp_user_data and \
            'keyring' in tmp_user_data and 'eddsa' in tmp_user_data['keyring']):

        res = generate_keypair(user_data)
        # Update data in user data
        user_data['seed'] = res['seed']
        user_data['eddsa_public_key'] = res['eddsa_public_key']
        user_data['keyring'] = {}
        user_data['keyring']['eddsa'] = res['keyring']['eddsa']

        with open(file,'w') as f:
            # save the entire data structure, not just one user
            json.dump(users_data, f)
    else:
        print(f"Keypair available from file for {user_data['name']}")
        user_data['seed'] = tmp_user_data['seed']
        user_data['eddsa_public_key'] = tmp_user_data['eddsa_public_key']
        user_data['keyring'] = {}
        user_data['keyring']['eddsa'] = tmp_user_data['keyring']['eddsa']



In [ ]:
# Read the keypair
endpt_filename = get_filename('cred_users.json')

read_keypair(endpt_filename, users_data, 'one')
read_keypair(endpt_filename, users_data, 'two')

In [ ]:
# Create the person using their public key
DEBUG_create_Person = False
def create_Person(name, username, email, eddsaPublicKey, endpoint, newPerson=True):

    if newPerson:
        variables = {
        "person": {
            "name": name,
            "user": username,
            "email": email,
            "eddsaPublicKey": eddsaPublicKey
            }
        }

        payload = {
          "query": """mutation ($person: PersonCreateParams!){
            createPerson(person: $person)
            {
                agent{
                    ...agent
                }
            }
           }""" + AGENT_FRAG,
          "variables": json.dumps(variables)
        }
    else:
        variables = {
          "email": email,
          "eddsaPublicKey": eddsaPublicKey
        }

        payload = {
          "query": """query ($email:String!, $eddsaPublicKey: String!){
              personCheck(email:$email, eddsaPublicKey:$eddsaPublicKey){
                id
              }
        }""",
          "variables": json.dumps(variables)
        }
        
#     print(json.dumps(payload, indent=2))

    # Temporarily: we need a key to create a person, before email authentication is implemented
    file = '.credentials.json'
    assert os.path.isfile(file)

    with open(file) as f:
        data = json.load(f)
        SECRET_KEY = data['key']
        
    res = requests.post(endpoint, json=payload, headers={'zenflows-admin': SECRET_KEY})
    
    if DEBUG_create_Person:
        print("Payload")
        print(payload)

        print("Headers")
        print(headers)

        print("Response")
        print(json.dumps(res, indent=2))

    result = res.json()

    if DEBUG_create_Person:
        print("Result")
        print(json.dumps(result, indent=2))

    
    if "errors" in result and len(result['errors']) > 0:
        for err in result['errors']:
            if err['message'] == "user: [\"has already been taken\"]":
                return create_Person(name, username, email, eddsaPublicKey, endpoint, newPerson=False)

    if newPerson:
        user_id = result['data']['createPerson']['agent']['id']
    else:
        user_id = result['data']['personCheck']['id']
    
    return user_id

In [ ]:
# Read the ID of the person from file or create a new person
def get_id_person(file, users_data, user, endpoint):
    user_data = users_data[f'{user}']
    
    if 'id' in user_data:
        print(f"Id available for {user_data['name']}")
        return
    
    if os.path.isfile(file):
        with open(file,'r') as f:
                tmp_users_data = json.loads(f.read())
                tmp_user_data = tmp_users_data[f'{user}']
    else:
        tmp_user_data = {}

    if not 'id' in tmp_user_data:
        user_data['id'] = create_Person(user_data['name'], user_data['username'], user_data['email'], user_data['eddsa_public_key'], endpoint)
#         print(json.dumps(user_data, indent=2))
        with open(file,'w') as f:
            json.dump(users_data, f)
    else:
        print(f"Id available from file for {user_data['name']}")
        user_data['id'] = tmp_user_data['id']
    

In [ ]:
# read or get id of the person
endpt_filename = get_filename('cred_users.json')

get_id_person(endpt_filename, users_data, 'one', endpoint=ENDPOINT)
get_id_person(endpt_filename, users_data, 'two', endpoint=ENDPOINT)

In [ ]:
# sign and send each request now that we have a registered public key
DEBUG_send_signed = False

def send_signed(query, variables, username, eddsa, endpoint):

    sign_script = """
    Scenario eddsa: sign a graph query
    Given I have a 'base64' named 'gql'
    Given I have a 'keyring'
    # Fix Apollo's mingling with query string
    When I remove spaces in 'gql'
    and I compact ascii strings in 'gql'
    When I create the eddsa signature of 'gql'
    And I create the hash of 'gql'
    Then print 'eddsa signature' as 'base64'
    Then print 'gql' as 'base64'
    Then print 'hash' as 'hex'
    """
    
    zenKeys = stringify({
        "keyring": {
            "eddsa": eddsa
        }
    })

    payload = {"query": query, "variables": variables}

    zenData = {
        "gql": base64.b64encode(bytes(json.dumps(payload), 'utf-8')).decode('utf-8')
    }

    zenData_str = stringify(zenData)
    
    try:
        result = zenroom.zencode_exec(sign_script, keys=zenKeys, data=zenData_str)
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

    res_json = json.loads(result.output)

    # Reset the headears
    headers = {}
    headers['content-type'] = 'application/json'

    headers['zenflows-sign'] = res_json['eddsa_signature']   
    headers['zenflows-user'] = username
    headers['zenflows-hash'] = res_json['hash']
    
    r = requests.post(endpoint, json=payload, headers=headers)

    res = r.json()

    if DEBUG_send_signed:
        print("Payload")
        print(payload)

        print("zenData")
        print(zenData)

        print("Zenroom result")
        print(result)

        print("Generated signature")
        print(json.dumps(res_json, indent=2))

        print("Headers")
        print(headers)

        print("Response")
        print(json.dumps(res, indent=2))
    
    return res

In [ ]:
# Read the location id from file or generate it by calling the back-end
DEBUG_get_location_id = False
def get_location_id(file, user_data, locs_data, user, endpoint):

    # this should not be possible since we initialize the data, but anyway
    if not f'{user}' in locs_data:
        locs_data[f'{user}'] = {}
        print("Warning this should not happen")

    loc_data = locs_data[f'{user}']

    if 'id' in loc_data:
        print(f"Location id available for {loc_data['name']}")
        return

    # check we already have a location file with an id
    if os.path.isfile(file):
        with open(file,'r') as f:
                temp_locs_data = json.loads(f.read())
                temp_loc_data = temp_locs_data[f'{user}']
                
    else:
        temp_loc_data = {}


    if 'id' not in temp_loc_data:
        # Register location
        # Produce the query and variables vars to be signed
        variables = {
            "location": {
                "name": loc_data['name'],
                "alt": 0,
                "lat": loc_data['lat'],
                "long": loc_data['long'],
                "mappableAddress": loc_data['addr'],
                "note": loc_data['note']
            }
        }

        query = """mutation($location: SpatialThingCreateParams!) {
                createSpatialThing(spatialThing: $location) {
                    spatialThing {
                        id
                    }
                }
            }"""

        res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)

    if DEBUG_get_location_id:
        print("Query")
        print(query)

        print("Variables")
        print(variables)

        print("Response")
        print(json.dumps(res, indent=2))

    result = res.json()

    if DEBUG_get_location_id:
        print("Result")
        print(json.dumps(result, indent=2))

        # save the id in the location data
        loc_data['id'] = res['data']['createSpatialThing']['spatialThing']['id']
        # reference the location in the user data
        loc_data['user_id'] = user_data['id']

        # save data with id
        with open(file,'w') as f:
            # Save the entire location data, not just the user one
            json.dump(locs_data, f)
    else:
        print(f"Location id available in file for {loc_data['name']}")
        loc_data['id'] = temp_loc_data['id']

In [ ]:
# Read of get the location id
endpt_filename = get_filename('loc_users.json')

get_location_id(endpt_filename, users_data['one'], locs_data, 'one', endpoint=ENDPOINT)
get_location_id(endpt_filename, users_data['two'], locs_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Read the unit id from file or generate it by calling the back-end
def get_unit_id(file, user_data, units_data, name, label, symbol, endpoint):
    
    if name in units_data and 'id' in units_data[f'{name}']:
        print(f"Unit {name} available")
        return

    # check we already have a unit file with an id
    if os.path.isfile(file):
        with open(file,'r') as f:
                temp_units_data = json.loads(f.read())
    else:
        temp_units_data = {}

    if not (name in temp_units_data and 'id' in temp_units_data[f'{name}']):

        # Produce the query and variables vars to be signed
        variables = {
                    "unit": {
                            "label": label,
                            "symbol": symbol
                            }
                }


        query = """mutation($unit:UnitCreateParams!) {
                createUnit(unit: $unit) {
                    unit {
                        id
                    }
                }
              }"""

        res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
#         print(res)

        # save the unit info
        units_data[f'{name}'] = {}
        units_data[f'{name}']['label'] = label
        units_data[f'{name}']['symbol'] = symbol
        units_data[f'{name}']['id'] = res['data']['createUnit']['unit']['id']

        # save data with id
        with open(file,'w') as f:
            json.dump(units_data, f)
    else:
        print(f"Unit available in file for {temp_units_data[f'{name}']}")
        units_data[f'{name}'] = {}
        units_data[f'{name}']['label'] = temp_units_data[f'{name}']['label']
        units_data[f'{name}']['symbol'] = temp_units_data[f'{name}']['symbol']
        units_data[f'{name}']['id'] = temp_units_data[f'{name}']['id']
        
        

In [ ]:
# Get the ids of all units
endpt_filename = get_filename('units_data.json')

get_unit_id(endpt_filename, users_data['two'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(endpt_filename, users_data['two'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(endpt_filename, users_data['two'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(endpt_filename, users_data['one'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)


In [ ]:
# Read the resource specification id or create a new one if not available
def get_resource_spec_id(file, user_data, res_spec_data, name, note, classification, default_unit_id, endpoint):

    if name in res_spec_data and 'id' in res_spec_data[f'{name}']:
        print(f"Specification {name} available")
        return

    # check we already have a unit file with an id
    if os.path.isfile(file):
        with open(file,'r') as f:
                temp_res_spec_data = json.loads(f.read())
    else:
        temp_res_spec_data = {}

    if not (name in temp_res_spec_data and 'id' in temp_res_spec_data[f'{name}']):

        # Produce the query and variables vars to be signed
        variables = {
            "resourceSpecification": {
                "defaultUnitOfResource": default_unit_id,
                "name": name,
                "note": note,
                "resourceClassifiedAs": classification
            }
        }


        query = """mutation ($resourceSpecification:ResourceSpecificationCreateParams!){
                    createResourceSpecification(resourceSpecification:$resourceSpecification){
                        resourceSpecification{
                            id,
                            name
                        }
                    }
                }"""

        res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
#         print(res)

        # save the unit info
        res_spec_data[f'{name}'] = {}
        res_spec_data[f'{name}']['note'] = note
        res_spec_data[f'{name}']['classification'] = classification
        res_spec_data[f'{name}']['defaultUnit'] = default_unit_id
        res_spec_data[f'{name}']['id'] = res['data']['createResourceSpecification']['resourceSpecification']['id']

        # save data with id
        with open(file,'w') as f:
            json.dump(res_spec_data, f)
    else:
        print(f"Specification available in file for {temp_units_data[f'{name}']}")
        res_spec_data[f'{name}'] = {}
        res_spec_data[f'{name}']['note'] = temp_res_spec_data[f'{name}']['note']
        res_spec_data[f'{name}']['classification'] = temp_res_spec_data[f'{name}']['classification']
        res_spec_data[f'{name}']['defaultUnit'] = temp_res_spec_data[f'{name}']['defaultUnit']
        res_spec_data[f'{name}']['id'] = temp_res_spec_data[f'{name}']['id']
        



In [ ]:
# Read all the resource specifications
endpt_filename = get_filename('res_spec_data.json')

name = 'soap'
note = 'Specification for soap to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q34396'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'water'
note = 'Specification for water to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q283'
default_unit_id = units_data['volume']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton'
note = 'Specification for cotton to be used to sew the gowns'
classification = 'https://www.wikidata.org/wiki/Q11457'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'gown'
note = 'Specification for gowns'
classification = 'https://www.wikidata.org/wiki/Q89990310'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'surgical_operation'
note = 'Specification for surgical operations'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = units_data['time']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# Create the resource by calling the back-end
DEBUG_create_resource = False
def create_resource(user_data, res_data, res_spec_data, amount, endpoint):
    
    provider = user_data['id']
    receiver = user_data['id']
    # Get the unit from the spec, no need to pass it     
    unit_id = [specs['defaultUnit'] for name, specs in res_spec_data.items() \
               if specs['id'] == res_data['spec_id']][0]
    
    # Produce the query and variables vars to be signed
    # Getting the current date and time
    ts = datetime.now(timezone.utc).isoformat()

    variables = {
        "event": {
            "note": "update event",
            "action": "raise",
            "provider": provider, 
            "receiver": receiver,
            "hasPointInTime" : ts,
            "resourceQuantity": {
              "hasUnit": unit_id,
              "hasNumericalValue": amount 
            },
            "resourceConformsTo": res_data['spec_id']
        },
        "newInventoriedResource": { 
            "name": res_data['name'],
            "trackingIdentifier": res_data['res_ref_id']
        }
    }

    query = """mutation($event:EconomicEventCreateParams!, $newInventoriedResource:EconomicResourceCreateParams) {
                createEconomicEvent(event:$event, newInventoriedResource:$newInventoriedResource) {
                    economicEvent {
                        id
                        provider {
                            ...agent
                        }
                        resourceQuantity {
                            ...quantity
                        }
                        resourceInventoriedAs {
                            ...resource
                      }
                    }
                }
            }""" + AGENT_FRAG + QUANTITY_FRAG + RESOURCE_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    if DEBUG_create_resource:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)

    # save the unit info
    res_data['id'] = res['data']['createEconomicEvent']['economicEvent']['resourceInventoriedAs']['id']
    
    return res['data']['createEconomicEvent']['economicEvent']['id'], ts




In [ ]:
# Create the resource by calling the back-end
DEBUG_reduce_resource = False
def reduce_resource(user_data, res_data, res_spec_data, amount, endpoint):
    
    provider = user_data['id']
    receiver = user_data['id']
    # Get the unit from the spec, no need to pass it     
    unit_id = [specs['defaultUnit'] for name, specs in res_spec_data.items() \
               if specs['id'] == res_data['spec_id']][0]
    
    # Produce the query and variables vars to be signed
    # Getting the current date and time
    ts = datetime.now(timezone.utc).isoformat()

    variables = {
        "event": {
            "note": "update event",
            "action": "lower",
            "provider": provider, 
            "receiver": receiver,
            "hasPointInTime" : ts,
            "resourceInventoriedAs" : res_data['id'],
            "resourceQuantity": {
              "hasUnit": unit_id,
              "hasNumericalValue": amount 
            },
            "resourceConformsTo": res_data['spec_id']
        }
    }

    query = """mutation($event:EconomicEventCreateParams!, $newInventoriedResource:EconomicResourceCreateParams) {
                createEconomicEvent(event:$event, newInventoriedResource:$newInventoriedResource) {
                    economicEvent {
                        id
                        provider {
                            ...agent
                        }
                        resourceQuantity {
                            ...quantity
                        }
                        resourceInventoriedAs {
                            ...resource
                      }
                    }
                }
            }""" + AGENT_FRAG + QUANTITY_FRAG + RESOURCE_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    if DEBUG_reduce_resource:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)

   
    return res['data']['createEconomicEvent']['economicEvent']['id'], ts


In [ ]:
# Wrapper for the resource creation
def get_resource(res_data, res_spec_data, res_name, user_data, event_seq, amount, endpoint):
    
#     set_trace()
    res_data[f'{res_name}_res'] = {}
    cur_res = res_data[f'{res_name}_res']

    rnd = random.randint(0, 10000)
    cur_res['res_ref_id'] = f'{res_name}-{rnd}'
    cur_res['name'] = res_name
    cur_res['spec_id'] = res_spec_data[f'{res_name}']['id']


    event_id, ts = create_resource(user_data, cur_res, res_spec_data, amount, endpoint)

    event_seq.append({'ts': ts, 'event_id':event_id, 'action' : 'raise', 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

res_name = 'soap'
amount = 100

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'water'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'cotton'
amount = 20

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
# Create a process by calling the back-end
def create_process(cur_process, user_data, endpoint):

    # Produce the query and variables vars to be signed
    # Getting the current date and time
    ts = datetime.now(timezone.utc).isoformat()

    variables = {
      "process": {
        "name": cur_process['name'],
        "note": cur_process['note'],
        "hasBeginning": ts,
        "hasEnd": ts
      }
    }

    query = """mutation($process:ProcessCreateParams!) {
        createProcess(process: $process) {
            process {
                id
            }
        }
    }"""

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
#     print(res)

    # save the unit info
    cur_process['id'] = res['data']['createProcess']['process']['id']

In [ ]:
# Wrapper for process creation
def get_process(process_name, process_data, note, user_data, endpoint):

#     name = process_name.replace(' ', '_')

    process_data[f'{process_name}'] = {}

    cur_process = process_data[f'{process_name}']

    cur_process['name'] = process_name
    cur_process['note'] = note
    

    create_process(cur_process, user_data, endpoint)    

In [ ]:
# Create the process that wraps sewing of the gown (its creation)
# print(process_data)
process_name = 'Sew_gown'
user_data = users_data['one']
note = f"Sew gown process performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# Create the process that wraps using the gown in the hospital and make it dirty
process_name = 'Use_gown'
user_data = users_data['one']
note = f"Use gown process performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# Create the process that includes cleaning the gown
process_name = 'Clean_gown'
user_data = users_data['two']
note = f"Clean gown process performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
DEBUG_create_event = False
# This function implements all actions != transfer actions
def create_event(user_data, action, note, amount, process, res_spec_data, endpoint, \
                 existing_res=None, new_res=None, effort_spec=None):

    if not action in SUPPORTED_ACTIONS:
        print(f"We do not support {action} yet")
        assert 1 == 2

    ts = datetime.now(timezone.utc).isoformat()
    variables = {
        "event": {
            "action": action,
            "note": note,
            "provider": user_data['id'],
            "receiver": user_data['id'],
            "hasPointInTime" : ts
        }
    }


    if action in ['use', 'work']:
        # If action is work or use then the quantity is about the action
        variables['event']['effortQuantity'] = {}
        var_obj = variables['event']['effortQuantity']
        var_obj['hasUnit'] = effort_spec['unit_id']
    else:
        # If action is not work then the quantity is about the resource
        variables['event']['resourceQuantity'] = {}
        var_obj = variables['event']['resourceQuantity']
        # find the unit from the resource's specification
        if action in ['produce']:
            _res = new_res
        else:
            _res = existing_res
        var_obj['hasUnit'] = [specs['defaultUnit'] for name, specs in res_spec_data.items() \
                              if specs['id'] == _res['spec_id']][0]

    
    var_obj['hasNumericalValue'] = amount


    if action in IN_PR_ACTIONS:
        # These actions are input to a process
        variables['event']['inputOf'] = process['id']
    elif action in OUT_PR_ACTIONS:
        # These actions are output of a process
        variables['event']['outputOf'] = process['id']
        
    if action in ['accept', 'cite', 'consume', 'modify', 'use']:
        # These actions require a resource id to act upon
        variables['event']['resourceInventoriedAs'] = existing_res['id']
        
    if action in ['work']:
        # Need to provide the specification of the type of work
        variables['event']['resourceConformsTo'] = effort_spec['spec_id']
        
    if action in ['produce']:
        variables['newInventoriedResource'] = {};
        variables['newInventoriedResource']['name'] = new_res['name']
        variables['newInventoriedResource']['trackingIdentifier'] = new_res['res_ref_id']
        variables['event']['resourceConformsTo'] = new_res['spec_id']
        

    
    # Define the fields for the GraphQL response

    response = """economicEvent {
                        id
                        provider {
                            ...agent
                        }
                        resourceQuantity {
                            ...quantity
                        }
                        toResourceInventoriedAs {
                            ...resource
                        }
                        resourceInventoriedAs {
                            ...resource
                        }
                    }"""
                    
    if action in ['produce']:
        query = f"""
        mutation($event:EconomicEventCreateParams!, $newInventoriedResource: EconomicResourceCreateParams) {{
                createEconomicEvent(event:$event, newInventoriedResource:$newInventoriedResource) {{
                    {response}
                }}
            }}"""
    else:
        query = f"""
        mutation($event:EconomicEventCreateParams!) {{
                createEconomicEvent(event:$event) {{
                    {response}
                }}
            }}"""

    query = query + AGENT_FRAG + QUANTITY_FRAG + RESOURCE_FRAG
    # assert False
    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)

    if DEBUG_create_event:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)

    if action in ['produce']:
        # save the id of the new resource
        new_res['id'] = res['data']['createEconomicEvent']['economicEvent']['resourceInventoriedAs']['id']

    return res['data']['createEconomicEvent']['economicEvent']['id'], ts


In [ ]:
# Define event consume for the gown creation
action = 'consume'
event_note='consume cotton for sewing'
amount = 10
cur_pros = process_data['Sew_gown']
cur_res = res_data['cotton_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Define event produce for the gown creation
action = 'produce'
event_note='produce gown'
amount = 1
cur_pros = process_data['Sew_gown']

res_data['gown_res'] = {
    "res_ref_id": f'gown-{random.randint(0, 10000)}',
    "name": 'gown',
    "spec_id": res_spec_data['gown']['id']
}
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Update the id of the resource in case of transfer
def update_id(res, new_id):
    if not 'previous_ids' in res:
        res['previous_ids'] = []
    res['previous_ids'].append(res['id'])
    res['id'] = new_id

# This function implements all transfer actions
# NOTE: only tested with "transfer-custody"
DEBUG_make_transfer = False

def make_transfer(provider_data, action, note, receiver_data, amount, existing_res, locs_data, res_spec_data, endpoint):

    ts = datetime.now(timezone.utc).isoformat()

    variables = {
        "event": {
            "note": note,
            "action": action,
            "provider": provider_data['id'], 
            "receiver": receiver_data['id'], 
            "resourceInventoriedAs": existing_res['id'],
            "hasPointInTime": ts,
            "atLocation": [values['id'] for key, values in locs_data.items() \
                              if values['user_id'] == receiver_data['id']][0],
            "resourceQuantity": {
              "hasUnit": [values['defaultUnit'] for key, values in res_spec_data.items() \
                              if values['id'] == existing_res['spec_id']][0], 
              "hasNumericalValue": amount 
            }
        },
        "newInventoriedResource": {
            "name" : existing_res['name']
        }
    }
    
    query = """mutation($event:EconomicEventCreateParams!, $newInventoriedResource: EconomicResourceCreateParams) {
                createEconomicEvent(event:$event, newInventoriedResource:$newInventoriedResource) {
                    economicEvent {
                        id
                        provider {
                            ...agent
                        }
                        resourceQuantity {
                            ...quantity
                        }
                        toResourceInventoriedAs { 
                            ...resource
                        }
                        resourceInventoriedAs {
                            ...resource
                        }
                    }
                }
            }""" + AGENT_FRAG + QUANTITY_FRAG + RESOURCE_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)

    if DEBUG_make_transfer:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    transferred_id = res['data']['createEconomicEvent']['economicEvent']['toResourceInventoriedAs']['id']

    update_id(existing_res, transferred_id)

    return res['data']['createEconomicEvent']['economicEvent']['id'], ts



In [ ]:
# Transfer the gown from the owner/leaser to the hospital
note='Transfer gowns to hospital'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['two'], action, note, users_data['one'], amount, cur_res,  locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Work with the gown to perform surgery
action = 'work'
event_note='work perform surgery'
amount = 80
cur_pros = process_data['Use_gown']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['surgical_operation']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['surgical_operation']['id']

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
    


In [ ]:
# Use the gown to perform surgery and as a consequence make it dirty
action = 'accept'
event_note='accept use for surgery'
amount = 1
cur_pros = process_data['Use_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})
    


In [ ]:
# Modify the gown and make it dirty as a consequence of being used
action = 'modify'
event_note='modify dirty after use'
amount = 1
cur_pros = process_data['Use_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the gown to the leaser for cleaning
note='Transfer gowns to cleaner'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['one'], action, note, users_data['two'], amount, cur_res,  locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept the gown for washing
action = 'accept'
event_note='accept gowns to be cleaned'
amount = 1
cur_pros = process_data['Clean_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume water for washing
action = 'consume'
event_note='consume water for the washing'
amount = 25
cur_pros = process_data['Clean_gown']
cur_res = res_data['water_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# consume soap for washing
action = 'consume'
event_note='consume soap for the washing'
amount = 50
cur_pros = process_data['Clean_gown']
cur_res = res_data['soap_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# modify the gown that is now clean
action = 'modify'
event_note='modify clean after washing'
amount = 1
cur_pros = process_data['Clean_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
show_data()

In [ ]:
DEBUG_show_resource = False
def show_resource(user_data, id, endpoint):

    variables = {
        "id": id
    }
    
    query = """query($id:ID!){
          economicResource(id:$id){
            ...resource    
          }
        }
    """ + RESOURCE_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)

    if DEBUG_show_resource:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    return res


In [ ]:
DEBUG_show_proposal = False
def show_proposal(user_data, id, endpoint):

    variables = {
        "id": id
    }
    
    query = """query($id:ID!){
      proposal(id:$id){
        name
        note
        id
        primaryIntents {
          resourceInventoriedAs {
            ...resource
          }
        }
      }
    }""" + RESOURCE_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)

    if DEBUG_show_resource:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    return res


In [ ]:
# example of using the above functions
# show_resource(users_data['one'], '061Z9FX8M4BJ3M6JMB7MW7VSER', endpoint=ENDPOINT)
# show_proposal(users_data['one'], '061Z9FX9H8PT0JF16KP25VS8R0', endpoint=ENDPOINT)


In [ ]:
DEBUG_trace_query = False

def trace_query(id, user_data, endpoint):

    variables = {
        "id": id
    }
    
    query = """query($id:ID!) {
    economicResource(id:$id) {
        trace {
          __typename
          ... on EconomicEvent {
            id
            action {id}
            provider {...agent}
            receiver {...agent}
            resourceConformsTo {id name note}
            resourceInventoriedAs {...resource}
            resourceQuantity {...measure}
          }
          ... on EconomicResource {...resource}
          ... on Process {id name note}
        }
      }
    }

    fragment measure on Measure {
      hasNumericalValue
      hasUnit {id label symbol}
    }

    fragment agent on Agent {
      id name
    }

    fragment resource on EconomicResource {
      id name note
      primaryAccountable {...agent}
      custodian {id name}
      onhandQuantity {...measure}
      accountingQuantity {...measure}
      classifiedAs
    }
    """

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    
    if DEBUG_trace_query:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    return res['data']['economicResource']['trace']

In [ ]:
BANNER = "#" * 80

def get_nodes(item, assigned):
    if item['id'] in assigned:
        assigned[item['id']]['count'] += 1
    else:
        assigned[item['id']] = {
            'count': 1,
            'type' : item['type'],
            'name' : item['name'],
        }

    nr_ch = len(item['children'])

    for ch in range(nr_ch):
        ch_dpp = item['children'][ch]
        get_nodes(ch_dpp, assigned)
    
def check_duplicates(trace, events, assigned):
    print(BANNER)
    print("Check whether there are any duplicated trace items")
    item_ids = {}
    for j, item in enumerate(trace):
        if not item['id'] in item_ids:
            item_ids[item['id']] = 1
        else:
            item_ids[item['id']] += 1
            print(f"Item {item['id']} of type {item['__typename']} at pos {j} is a duplicate")

    print(BANNER)
    print("Check whether there are any duplicated events")
    events_ids = {}
    for i, event in enumerate(events):
        if not event['event_id'] in events_ids:
            events_ids[event['event_id']] = 1
        else:
            events_ids[event['event_id']] += 1
            print(f"Event {event['event_id']} with action {event['action']} at pos {i} is a duplicate")

    print(BANNER)
    print("Check whether there are any duplicated in dpp")
    for id in assigned.keys():
        el = assigned[id]
        if el['count'] > 1:
            print(f"Element {el['name']} with id {id} with type {el['type']} has {el['count']-1} duplicates")
    

def check_trace_events(trace, events):
    # we reverse the events to have last in first out 
    events.reverse()

    print(BANNER)
    print("Where are trace items in the events?")
    for j, item in enumerate(trace):
        found = False
        name = item['name'] if 'name' in item else item['action']['id']
        pref = f"trace item {name} id: {item['id']} of type {item['__typename']}"
        for i, event in enumerate(events):
            event_id = event['event_id']
            if item['id'] == event_id:
                print(f"{pref} at pos {j} found at pos {i}")
                found = True
        if not found:
            print(f"NOT FOUND: {pref}")

    print(BANNER)
    print("Where are events in the trace?")
    for i, event in enumerate(events):
        found = False
        event_id = event['event_id'] if 'event_id' in event else event['process_id']
        name = event['action']
        pref = f'Event {name} with id {event_id}'
        for j,item in enumerate(trace):
            if item['id'] == event_id:
                print(f'{pref} pos {i} found at pos {j}')
                found = True
        if not found:
            print(f"NOT FOUND: {pref}")

def check_trace_dpp(trace, assigned):
    print(BANNER)
    print("Are trace items in the dpp?")
    for j, item in enumerate(trace):
        found = False
        if item['id'] not in assigned:
            name = item['name'] if 'name' in item else item['action']['id']
            pref = f"trace item {name} id: {item['id']} of type {item['__typename']}"
            print(f"NOT FOUND: {pref} at pos {j}")

    print(BANNER)
    print("Are dpp elements in the trace?")
    for id in assigned.keys():
        el = assigned[id]
        pref = f"Element {el['name']} with id {id} of type {el['type']}"
        found = False
        for j,item in enumerate(trace):
            if item['id'] == id:
                print(f"{pref} found at pos {j}")
                found = True
        if not found:
            print(f"NOT FOUND: {pref}")

def check_betrace(tot_dpp, be_dpp):
    if tot_dpp['id'] == be_dpp['node']['id']:
        nr_ch = len(tot_dpp['children'])
        nr_ch_be = len(be_dpp['children'])
        if nr_ch == nr_ch_be:
            if nr_ch == 0:
                return
            elif nr_ch == 1:
                check_betrace(tot_dpp['children'][0], be_dpp['children'][0])
            else:
                for ch in tot_dpp['children']:
                    found = False
                    for ch_be in be_dpp['children']:
                        if ch['id'] == ch_be['node']['id']:
                            found = True
                            check_betrace(ch, ch_be)
                    if not found:
                        print(f"Children {tot_dpp['id']} and {be_dpp['node']['id']} differ in ids")
                        break
                return
        else:
            print(f"Children of id {tot_dpp['id']} differ in number")
            print(f"Children of back-end")
            for ch in be_dpp['children']:
                name = ch['node']['name'] if 'name' in ch['node'] else ch['node']['action_id']
                print(f"Name: {name}, id {ch['node']['id']}")
            print(f"Children of front-end")
            for ch in tot_dpp['children']:
                print(f"Name: {ch['name']}, id {ch['id']}")
            
    else:
        print(f"{tot_dpp['id']} different from {be_dpp['node']['id']}")


def check_traces(trace, events, tot_dpp, be_dpp):
    assigned = {}
    get_nodes(tot_dpp[0], assigned)
    print(BANNER)
    print(f'nr trace: {len(trace)}, nr events: {len(events)}, nr dpp: {len(assigned)}')
    check_duplicates(trace, events,assigned)
    check_trace_events(trace, events)
    check_trace_dpp(trace, assigned)
    print(BANNER)
    print("Are back-end and my trace the same?")

    check_betrace(tot_dpp[0], be_dpp)


In [ ]:
VERBOSE = True
def fill_loc(a_dpp_item, item, field):
    if item[field] == None:
        return
    loc_item = item[field]
    a_dpp_item[field] = {}
    a_dpp_item[field]['id'] = loc_item['id']
    a_dpp_item[field]['name'] = loc_item['name']
    a_dpp_item[field]['alt'] = loc_item['alt']
    a_dpp_item[field]['lat'] = loc_item['lat']
    a_dpp_item[field]['long'] = loc_item['long']                    
    a_dpp_item[field]['mappableAddress'] = loc_item['mappableAddress']                    
    a_dpp_item[field]['note'] = loc_item['note']                                    
    
def fill_quantity(a_dpp_item, item, field):
    if item[field] != None:
        a_dpp_item[field] = f"{item[field]['hasNumericalValue']} ({item[field]['hasUnit']['symbol']})"

    
def fill_event(a_dpp_item, item):
    assert item['__typename'] == 'EconomicEvent'
    a_dpp_item['id'] = item['id']
    a_dpp_item['name'] = item['action']['id']
    a_dpp_item['type'] = item['__typename']
#     set_trace()
    if VERBOSE:
        a_dpp_item['provider'] = item['provider']['name']
        a_dpp_item['receiver'] = item['receiver']['name']
        fill_loc(a_dpp_item, item, 'atLocation')
        fill_loc(a_dpp_item, item, 'toLocation')
        if 'effortQuantity' in item and item['effortQuantity'] != None:
            fill_quantity(a_dpp_item, item, 'effortQuantity')
        elif 'resourceQuantity' in item and item['resourceQuantity'] != None:
            fill_quantity(a_dpp_item, item, 'resourceQuantity')
#         elif 'resourceInventoriedAs' in item and item['resourceInventoriedAs'] != None:
#             fill_quantity(a_dpp_item, item['resourceInventoriedAs'], 'onhandQuantity')
        
def fill_res(a_dpp_item, item):
#     print(item['__typename'])
    assert item['__typename'] == 'EconomicResource'
    a_dpp_item['id'] = item['id']
    a_dpp_item['name'] = item['name']
    a_dpp_item['trackingIdentifier'] = item['trackingIdentifier']
    a_dpp_item['type'] = item['__typename']
    if VERBOSE:
        a_dpp_item['primaryAccountable'] = item['primaryAccountable']['name']
        a_dpp_item['custodian'] = item['custodian']['name']
        a_dpp_item['metadata'] = item['metadata']
        fill_loc(a_dpp_item, item, 'currentLocation')
        fill_quantity(a_dpp_item, item, 'accountingQuantity')
        fill_quantity(a_dpp_item, item, 'onhandQuantity')

def fill_process(a_dpp_item, item):
    assert item['__typename'] == 'Process'
    a_dpp_item['id'] = item['id']
    a_dpp_item['name'] = item['name']
    a_dpp_item['type'] = item['__typename']
    if VERBOSE:
        a_dpp_item['note'] = item['note']


In [ ]:

DEBUG_er_before = False

def er_before(id, user_data, dpp_children, depth, visited, endpoint):

    if depth > MAX_DEPTH:
        return
    depth += 1

    variables = {
        "id": id
    }
    
    query = """query($id:ID!) {
        economicResource(id:$id) {
            id
            name
            trackingIdentifier
            __typename
            metadata
            primaryAccountable {
              ...agent
            }
            custodian {
              ...agent
            }
            accountingQuantity {
                ...quantity
            }
            onhandQuantity{
                ...quantity
            }
            currentLocation {
                ...location
            }
            previous{
                __typename
                ... on EconomicEvent {
                    id
                    action {
                        id
                    }
                }
            }
        }
    }
    """ + LOCATION_FRAG + QUANTITY_FRAG + AGENT_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    
    if DEBUG_er_before:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    dpp_item = {}    
    fill_res(dpp_item, res['data']['economicResource'])
    dpp_item['children'] = []
    
    dpp_children.append(dpp_item)
    
    events = res['data']['economicResource']['previous']
    while events != []:
        # We get the first event
        event = events.pop(0)
        # This must be of type EconomicEvent since the call only returns that             
        assert event['__typename'] == "EconomicEvent"
        # We include the raise event as it can be reached from more branches
        # and it is assumed to be the starting point        
#         if not event['action']['id'] == 'raise':
        while event['id'] in visited:
#                 set_trace()
            if DEBUG_er_before:
                print(f"id {event['id']} already in visited")
            if events == []:
                return
            event = events.pop(0)
        visited[event['id']] = {}            
            
        ee_before(event['id'], user_data, dpp_item['children'], depth, visited, endpoint)         


In [ ]:
DEBUG_ee_before = False

def ee_before(id, user_data, dpp_children, depth, visited, endpoint):

    if depth > MAX_DEPTH:
        return
    depth += 1

    variables = {
        "id": id
    }
    
    query = """query($id:ID!) {
        economicEvent(id:$id) {
            id
            __typename
            action {
                id
                label
            }
            provider {
                ...agent
            }
            receiver {
                ...agent
            }
            atLocation {
                ...location
            }
            toLocation {
                ...location
            }
            effortQuantity {
                ...quantity
            }
            resourceQuantity {
                ...quantity
            }
            previous{
                __typename
                ... on  EconomicResource {
                    id
                    name
                }
                ... on EconomicEvent {
                    id
                    action {
                        id
                    }
                }
                ... on Process {
                    id
                    name
                }
            }
          }
        }
    """ + LOCATION_FRAG + QUANTITY_FRAG + AGENT_FRAG

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    
    if DEBUG_ee_before:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    dpp_item = {}    
    fill_event(dpp_item, res['data']['economicEvent'])
    dpp_item['children'] = []
    
    dpp_children.append(dpp_item)

    pf_items = res['data']['economicEvent']['previous']
    if DEBUG_ee_before:
        print("pf_items")
        print(pf_items)

    if pf_items == None:
        return
    if type(pf_items) is dict:
        pf_items = [pf_items]
    if pf_items != []:
        pf_item = pf_items[0]
        if pf_item['id'] in visited:
            print(f"id {pf_item['id']} already in visited")
            return

        if pf_item['__typename'] == "EconomicEvent":
            visited[pf_item['id']] = {}
            ee_before(pf_item['id'], user_data, dpp_item['children'], depth, visited, endpoint)
        if pf_item['__typename'] == "EconomicResource":
            er_before(pf_item['id'], user_data, dpp_item['children'], depth, visited, endpoint)
        if pf_item['__typename'] == "Process":
            visited[pf_item['id']] = {}
            pr_before(pf_item['id'], user_data, dpp_item['children'], depth, visited, endpoint)


In [ ]:
DEBUG_pr_before = False

def pr_before(id, user_data, dpp_children, depth, visited, endpoint):

    if depth > MAX_DEPTH:
        return
    depth += 1

    variables = {
        "id": id
    }
    
    query = """query($id:ID!) {
      process(id:$id) {
          id
          name
          note
          __typename
          previous{
            __typename
            ... on EconomicEvent {
                id
                action {
                    id
                }
            }
        }
      }
    }
    """

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    
    if DEBUG_pr_before:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)   

    dpp_item = {}    
    fill_process(dpp_item, res['data']['process'])
    dpp_item['children'] = []
    
    dpp_children.append(dpp_item)

    events = res['data']['process']['previous']
    if events != []:
        for event in events:
            # This must be of type EconomicEvent since the call only returns that
            assert event['__typename'] == "EconomicEvent"
            if event['id'] in visited:
                print(f"id {event['id']} already in visited")
                continue
            visited[event['id']] = {}
            ee_before(event['id'], user_data, dpp_item['children'], depth, visited, endpoint)


In [ ]:
DEBUG_get_ddp = True

def get_ddp(res_id, user_data, endpoint):

    variables = {
        "id": res_id
    }
    
    query = """query($id:ID!){
        economicResource(id: $id) {
            traceDpp
        }
    }
    """

    
    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'], endpoint)
    
    if DEBUG_get_ddp:
        print("Query")
        print(query)
        print("Variables")
        print(variables)
        print("Result")
        print(res)
        
    be_dpp = res['data']['economicResource']['traceDpp']
    
    return be_dpp


In [ ]:
trace_me = res_data['gown_res']['id']
# trace_me = '062HCB36KR2QRY5HPP2RETC9M4'
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = {}
er_before(trace_me, users_data['one'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_ddp(trace_me, user_data, endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['one'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
WRITE_FILES = False
if WRITE_FILES:
    # Writing dpp to file
    with open("isogown_trace.json", "w") as f:
        f.write(json.dumps(tot_dpp, indent=2))
    with open("isogown_trace.list.json", "w") as f:
        f.write(json.dumps(trace, indent=2))
    with open("isogown_trace.tree.json", "w") as f:
        f.write(json.dumps(be_dpp, indent=2))        
    with open("isogown_events.json", "w") as f:
        f.write(json.dumps(event_seq, indent=2))

In [ ]:
def make_sankey(srcs, trgs, lbls, vls, clr_nodes, clr_links):
    # data to dict, dict to sankey
    link = dict(source = srcs, target = trgs, value = vls, color=clr_links)
    node = dict(label = lbls, pad=10, thickness=5, color=clr_nodes)
    data = go.Sankey(link = link, node=node)
    # plot
    fig = go.Figure(data)
    fig.show()

In [ ]:
dict_node_colors = {'EconomicResource': '#e41a1c', 'EconomicEvent': '#377eb8', 'Process': '#4daf4a', 'Transfer': '#984ea3'}
dict_link_colors = {'EconomicResource': '#fb6a4a', 'EconomicEvent': '#92c5de', 'Process': '#99d8c9', 'Transfer': '#bcbddc'}
def calc_quantity(dpp_item):
    if 'onhandQuantity' in dpp_item:
        quantity = dpp_item['onhandQuantity']
    elif 'effortQuantity' in dpp_item:
        quantity = dpp_item['effortQuantity']
    else:
        quantity = '1 '
    quantity = int(quantity.split(' ')[0])
    quantity = max(quantity,1)
    return quantity
    
def vis_dpp(dpp_item, count, assigned, labels, targets, sources, values, color_nodes, color_links):
    name = dpp_item['name']
#     print(f"vis name: {name}, count: {count}")
    if dpp_item['type'] != "EconomicResource":
        if dpp_item['id'] in assigned:
            assigned[dpp_item['id']].append(count)
        else:
            assigned[dpp_item['id']] = [count]
    quantity = calc_quantity(dpp_item)
    labels.append(name)
    el_type = 'Transfer' if dpp_item['type'] == 'EconomicEvent' and 'transfer' in dpp_item['name'].lower() else dpp_item['type']
    color_nodes.append(dict_node_colors[el_type])
    nr_ch = len(dpp_item['children'])
    new_count = count + 1
    for ch in range(nr_ch):
        targets.append(count)
        sources.append(new_count)
        ch_dpp = dpp_item['children'][ch]
        el_type = 'Transfer' if ch_dpp['type'] == 'EconomicEvent' and 'transfer' in ch_dpp['name'].lower() else ch_dpp['type']
        color_links.append(dict_link_colors[el_type])
        if quantity == 0:
            set_trace()
        values.append(quantity)
#         values.append(1)
        new_count = vis_dpp(ch_dpp, new_count, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
    return new_count

def consol_trace(assigned, sources, targets):
    for key in assigned.keys():
        if len(assigned[key]) > 1:
            # print(key)
            vl0 = assigned[key][0]
            for vl_idx in range(1,len(assigned[key])):
                vl = assigned[key][vl_idx]
                sources = [i if i != vl else vl0 for i in sources]
                targets = [i if i != vl else vl0 for i in targets]
    return sources, targets


In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)
# make_sankey([0,0,1,2,2], [2,3,3,3,4], ['0','1','2','3','4'], [2,1,1,1,1], color_nodes, color_links)

Here we start some example visualisations

In [ ]:
label = ['Operating Expeditures', 
         'Public Safety', 
         'Engineering and Utilities', 
         'Community-Related Services', 
         'Corporate Support',
         'Police', 
         'Fire',
         'Utilities', 
         'Engineering Public Works',
         'Parks and Recreation', 
         'Arts, Culture, and Community Services',
         'Library', 
         'Development, Buildings, and Licensing',
         'Planning, Urban Design, and Sustainability', 
         'Other',
         'Corporate Support', 
         'Debt and Capital (Non-Utility)', 
         'Contingencies and Transfers']

color_node = ['#808B96', 
             '#EC7063', '#F7DC6F', '#48C9B0', '#AF7AC5',
             '#EC7063', '#EC7063',
             '#F7DC6F', '#F7DC6F',
             '#48C9B0', '#48C9B0', '#48C9B0', '#48C9B0', '#48C9B0', '#48C9B0',
             '#AF7AC5', '#AF7AC5', '#AF7AC5']

color_link = ['#EBBAB5', '#FEF3C7', '#A6E3D7', '#CBB4D5',
              '#EBBAB5', '#EBBAB5',
              '#FEF3C7', '#FEF3C7',
              '#A6E3D7', '#A6E3D7', '#A6E3D7', '#A6E3D7', '#A6E3D7', '#A6E3D7',
              '#CBB4D5', '#CBB4D5', '#CBB4D5']


source = [0, 0, 0, 0,
          1, 1,
          2, 2,
          3, 3, 3, 3, 3, 3,
          4, 4, 4]

target = [1, 2, 3, 4, 
          5, 6,
          7, 8, 
          9, 10, 11, 12, 13, 14, 
          15, 16, 17]

value = [484900, 468350, 355300, 306850, 
         339150, 145350, 
         371450, 96900, 
         129200, 80750, 48450, 48450, 32300, 16150, 
         113050, 129200, 64600]

In [ ]:
# data to dict, dict to sankey
link = dict(source = source, target = target, value = value, color=color_link)
node = dict(label = label, pad=15, thickness=5, color=color_node)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()